In [686]:
from lxml import etree, html
from lxml.html.clean import clean_html
import pandas as pd
from datetime import date, datetime
import requests
import re
import os
import shutil
from selenium import webdriver

In [687]:
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def pdf2txt(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text

# Mueve todos los fichero del directorio de trabajo a otro de archivo

In [688]:
source_dir = './DOCMs'
target_dir = './DOCMs_Anteriores'
    
file_names = os.listdir(source_dir)
    
for file_name in file_names:
    #shutil.move(os.path.join(source_dir, file_name), target_dir)
    # Evita que de error si el fichero que se mueve ya existe en dir destino
    try:
        os.remove(os.path.join(target_dir, file_name))
        shutil.move(os.path.join(source_dir, file_name), target_dir)
    except OSError:
        shutil.move(os.path.join(source_dir, file_name), target_dir)

# Recoge resumen diario del BOE de hoy

In [689]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)

print(today.strftime("%d/%m/%Y"))

Fecha de Hoy = 20210212
dia = 12
mes = 02
año = 2021
12/02/2021


In [690]:
URL_HTML_resumen =  "https://docm.jccm.es/portaldocm/cambiarBoletin.do?fecha=" + str(hoy)

In [691]:
URL_HTML_resumen

'https://docm.jccm.es/portaldocm/cambiarBoletin.do?fecha=20210212'

In [692]:
# carga página HTML y genera árbol

response = requests.get(URL_HTML_resumen)
sumario_HTML = html.fromstring(response.text)

## Recoge Nombre Secciones Sumario

secciones = sumario_HTML.xpath('//*[@class="cabeceraCategoria"]/text()')
secciones = [ re.sub('\r|\n|\t','', seccion) for seccion in secciones ]

In [693]:
secciones

['I.- DISPOSICIONES GENERALES',
 'II.- AUTORIDADES Y PERSONAL',
 'III.- OTRAS DISPOSICIONES Y ACTOS',
 'IV.- ADMINISTRACIÓN DE JUSTICIA',
 'V.- ANUNCIOS']

In [694]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

save_html(response.content, './DOCMs/Resumen-DOCM-' + hoy + '.html')

In [695]:
for seccion in sumario_HTML.xpath('//*[@class="cabeceraCategoria"]'):
    nombre_seccion = seccion.xpath('./text()')
    nombre_seccion = str(nombre_seccion[0]).strip()
    print(nombre_seccion)


I.- DISPOSICIONES GENERALES
II.- AUTORIDADES Y PERSONAL
III.- OTRAS DISPOSICIONES Y ACTOS
IV.- ADMINISTRACIÓN DE JUSTICIA
V.- ANUNCIOS


In [696]:
DOCM_sumarios = pd.DataFrame(columns=['item_Title','item_urlHTML','item_urlPDF'])

In [697]:
DOCM_sumarios

,item_Title,item_urlHTML,item_urlPDF


In [698]:
lista = []
for sumario in sumario_HTML.xpath('//*[@class="sumario"]'):
    lista.append(sumario.text_content().strip())

DOCM_sumarios['item_Title'] = lista

lista = []   
for link_HTML in sumario_HTML.xpath('//*[@title="Ver los datos detallados del documento"]'):
    link = link_HTML.xpath('./@href')
    #print(link)
    lista.append('https://docm.jccm.es/portaldocm' + str(link)[3:-2])

DOCM_sumarios['item_urlHTML'] = lista

lista = []   
for link_PDF in sumario_HTML.xpath('//div/a[@class="new-window"]'):
    link = link_PDF.xpath('./@href')
    #print(link)
    lista.append('https://docm.jccm.es/portaldocm' + str(link)[3:-2])

DOCM_sumarios['item_urlPDF'] = lista

In [699]:
DOCM_sumarios['item_Title'][0]

'Sanidad. Procedimiento Ordinario 119/2021. [NID 2021/1372]'

In [700]:
# Consolida las columnas Referencias_palabra y Referencias_texto en una única frase
# for i, row in DOCM_sumarios.iterrows():
#     # carga página HTML y genera árbol
#     response = requests.get(row['item_urlHTML'])
#     sumario_HTML = html.fromstring(response.text)
#     HTML = html.tostring(sumario_HTML)
# 
#     #NID = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[2]/td[2]/text()')
#     print(response.text)





In [701]:
print(DOCM_sumarios['item_urlHTML'][0])

https://docm.jccm.es/portaldocm/detalleDocumento.do?idDisposicion=1612960518482440526


In [702]:
# response = requests.get(DOCM_sumarios['item_urlHTML'][0])
# 
# sumario_HTML = html.fromstring(response.text)
# print(response.text)

In [703]:
### Necesita libreria Selenium para renderizar JS script

options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

driver.get(DOCM_sumarios['item_urlHTML'][0])
#print(driver.page_source)
response = driver.page_source

sumario_HTML = html.fromstring(response)
numero_diario = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[1]/td[2]/text()')
numero_pagina = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[3]/td[4]/text()')
NID = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[2]/td[2]/text()')
rango = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[5]/td[2]/text()')
organo_emisor = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[7]/td[2]/text()')

print(numero_diario[0].strip(), numero_pagina[0].strip(), NID[0].strip(), rango[0].strip(), organo_emisor[0].strip())
print()

driver.quit()

29/2021 5362 2021/1372 Edicto Tribunal Superior de Justicia de Castilla-La Mancha



In [742]:
### Recoge información de página de detalle con Selenium
### es necesario ya que la página se genera con un JS y
### hay que renderizarla con un headless web browser

options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

DOCM_sumarios['NID'] = ''
DOCM_sumarios['numero_diario'] = ''
DOCM_sumarios['numero_pagina'] = ''
DOCM_sumarios['rango'] = ''
DOCM_sumarios['organo_emisor'] = ''
DOCM_sumarios['Fecha_publicacion'] = ''

for i, row in DOCM_sumarios.iterrows():
    # carga página HTML y genera árbol
    driver.get(row['item_urlHTML'])
    #print(driver.page_source)
    response = driver.page_source
    
    sumario_HTML = html.fromstring(response)
    numero_diario = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[1]/td[2]/text()')
    numero_pagina = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[3]/td[4]/text()')
    NID = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[2]/td[2]/text()')
    rango = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[5]/td[2]/text()')
    organo_emisor = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[7]/td[2]/text()')
    item_itle = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[9]/td[2]/text()')

    #print(numero_diario[0].strip(), numero_pagina[0].strip(), NID[0].strip(), rango[0].strip(), organo_emisor[0].strip())

    DOCM_sumarios['NID'][i] = NID[0].strip()
    DOCM_sumarios['numero_diario'][i] = numero_diario[0].strip()
    DOCM_sumarios['numero_pagina'][i] = numero_pagina[0].strip()
    DOCM_sumarios['rango'][i] = rango[0].strip()
    DOCM_sumarios['organo_emisor'][i] = organo_emisor[0].strip()

    DOCM_sumarios['Fecha_publicacion'][i] = datetime.strptime(str(today),"%Y-%m-%d").date()

driver.quit()

In [743]:
DOCM_sumarios

,item_Title,item_urlHTML,item_urlPDF,NID,numero_diario,numero_pagina,rango,organo_emisor,Tags,Match_ASECORP_BBDD,Fecha_publicacion
0,Sanidad. Procedimiento Ordinario 119/2021. [NI...,https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1372,29/2021,5362,Edicto,Tribunal Superior de Justicia de Castilla-La M...,"[Resolución de 28 de enero de 2021, Resolución...",[],2021-02-12
1,"Régimen Local. Resolución de 05/02/2021, de la...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1231,29/2021,5364,Resolución,Viceconsejería de Administración Local y Coord...,"[Ley 7/1985, Real Decreto 128/2018, Real Decre...",[],2021-02-12
2,"Función Pública. Resolución de 02/02/2021, de ...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1109,29/2021,5365,Resolución,Dirección General de la Función Pública,"[Ley 40/2015, Ley 31/1995, Ley 12/2010, Real D...","[LEY 31/1995 ESP, RD 171/2004 ESP, RDG 2/2015 ...",2021-02-12
3,"Formación. Resolución de 05/02/2021, de la Dir...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1248,29/2021,5380,Resolución,Dirección General de la Función Pública,"[Ley 4/2011, Ley 40/2015, Ley 39/2015, Ley 40/...","[LEY 9/2017 ESP, RD 1771/1994 ESP]",2021-02-12
4,"Convenios. Resolución de 05/02/2021, de la Sec...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1250,29/2021,5504,Resolución,Secretaría General de la Presidencia,"[Ley 19/2013, Ley 19/2013, Ley 40/2015, Ley 40...",[],2021-02-12
...,...,...,...,...,...,...,...,...,...,...,...
63,"Anuncio de 01/02/2021, del Ayuntamiento de Poz...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1122,29/2021,5595,Anuncio,Ayuntamiento de Pozohondo (Albacete),[],[],2021-02-12
64,"Anuncio de 29/01/2021, del Ayuntamiento de La ...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1154,29/2021,5596,Anuncio,Ayuntamiento de La Villa de Don Fadrique (Toledo),[],[],2021-02-12
65,"Anuncio de 05/02/2021, de la Empresa Pública G...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1202,29/2021,5597,Anuncio,Empresa Pública Gestión Ambiental de Castilla-...,[],[],2021-02-12
66,"Anuncio de 05/02/2021, de la Empresa Pública G...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1203,29/2021,5598,Anuncio,Empresa Pública Gestión Ambiental de Castilla-...,[],[],2021-02-12


# Salva PDFs y Genera DF con datos Análisis de cada PDF

In [706]:
# Crea nueva columna vacía de tipo lista en tabla_analisis
#DOCM_sumarios['Referencias_completas'] = [[] for i in range(len(tabla_analisis))]
DOCM_sumarios['Tags'] = [[] for i in range(len(DOCM_sumarios))]
DOCM_sumarios['Match_ASECORP_BBDD'] = [[] for i in range(len(DOCM_sumarios))]


In [707]:
# Define expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
pattern = ['Ley [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+','Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Circular [0-9]+\/[0-9]+']

In [708]:
for i, row in DOCM_sumarios.iterrows():
    r = requests.get(row['item_urlPDF'])

    # Salva PDFs de enlaces a items
    filename = row['NID'].replace('/','-' )
    f = './DOCMs/' + 'DOCM_NID_' + filename + '.pdf'
    save_html(r.content, f)

    # Extrae texto de PDFs
    pdf_contents = pdf2txt(f)
    #print(pdf_contents)

    # Busca expresiones REGX coincidentes con Patrones definidos
    DOCM_sumarios['Tags'][i] = re.findall('|'.join(pattern), str(pdf_contents), flags=re.IGNORECASE)
    print(DOCM_sumarios['Tags'][i])

['Resolución de 28 de enero de 2021', 'Resolución de 06 de febrero de 2021']
['Ley 7/1985', 'Real Decreto 128/2018', 'Real Decreto 128/2018']
['Ley 40/2015', 'Ley 31/1995', 'Ley 12/2010', 'Real Decreto Legislativo 5/2015', 'Ley 4/2011', 'Real Decreto Legislativo 2/2015', 'Ley 4/2011', 'Ley 12/2010', 'Ley 12/2010', 'Ley 12/2010', 'Ley 31/1995', 'Ley 12/2010', 'Ley 31/1995', 'Ley 40/2015', 'Ley 40/2015']
['Ley 4/2011', 'Ley 40/2015', 'Ley 39/2015', 'Ley 40/2015', 'Ley 39/2015', 'Ley 40/2015', 'Ley 40/2015', 'Ley 40/2015', 'Ley 53/84', 'Ley 9/2017', 'Ley 9/2017', 'Ley 9/2017', 'Ley 9/2017', 'ley 39/2015', 'ley 40/2015', 'Ley 40/2015', 'Ley 39/2015', 'ley 39/2015', 'ley 40/2015', 'Ley 40/2015', 'Ley 39/2015', 'Ley 39/2015', 'Ley 40/2015', 'Ley 40/2015', 'Ley 39/2015', 'Ley 40/2015', 'Ley 39/2015', 'Ley 40/2015', 'Ley 39/2015', 'Ley 39/2015', 'Ley 30/1992', 'Ley 8/2019']
['Ley 19/2013', 'Ley 19/2013', 'Ley 40/2015', 'Ley 40/2015', 'Ley 19/2013', 'Real Decreto 922/2020', 'Ley 11/2003', 'Ley 

In [709]:
# Aplica expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
regex_result = []
[ regex_result.append(tag) for tags in DOCM_sumarios['Tags'] for tag in tags ]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [710]:
print(regex_result)

['Resolución de 28 de enero de 2021', 'Resolución de 06 de febrero de 2021', 'Ley 7/1985', 'Real Decreto 128/2018', 'Real Decreto 128/2018', 'Ley 40/2015', 'Ley 31/1995', 'Ley 12/2010', 'Real Decreto Legislativo 5/2015', 'Ley 4/2011', 'Real Decreto Legislativo 2/2015', 'Ley 4/2011', 'Ley 12/2010', 'Ley 12/2010', 'Ley 12/2010', 'Ley 31/1995', 'Ley 12/2010', 'Ley 31/1995', 'Ley 40/2015', 'Ley 40/2015', 'Ley 4/2011', 'Ley 40/2015', 'Ley 39/2015', 'Ley 40/2015', 'Ley 39/2015', 'Ley 40/2015', 'Ley 40/2015', 'Ley 40/2015', 'Ley 53/84', 'Ley 9/2017', 'Ley 9/2017', 'Ley 9/2017', 'Ley 9/2017', 'ley 39/2015', 'ley 40/2015', 'Ley 40/2015', 'Ley 39/2015', 'ley 39/2015', 'ley 40/2015', 'Ley 40/2015', 'Ley 39/2015', 'Ley 39/2015', 'Ley 40/2015', 'Ley 40/2015', 'Ley 39/2015', 'Ley 40/2015', 'Ley 39/2015', 'Ley 40/2015', 'Ley 39/2015', 'Ley 39/2015', 'Ley 30/1992', 'Ley 8/2019', 'Ley 19/2013', 'Ley 19/2013', 'Ley 40/2015', 'Ley 40/2015', 'Ley 19/2013', 'Real Decreto 922/2020', 'Ley 11/2003', 'Ley 19/2

In [711]:
## Elimina duplicados
boletin_flat_list = list(set(regex_result))

In [712]:
boletin_flat_list

['Ley 4/2007',
 'Ley 53/84',
 'Ley 24/2013',
 'Ley 12/2010',
 'Ley 47/2003',
 'Real Decreto 110/2015',
 'Ley 21/1992',
 'Real Decreto 833/1998',
 'Ley 9/2003',
 'Real Decreto 212/2002',
 'Resolución de 28 de enero de 2021',
 'Ley 37/2003',
 'Ley 34/2007',
 'Resolución de 14 de diciembre de 2020',
 'Ley 11/2003',
 'Ley 3/2004',
 'Ley 9/1999',
 'Ley 6/1985',
 'Ley 39/2015',
 'ley 39/2015',
 'Real Decreto 1093/1997',
 'Ley 8/1990',
 'Ley 7/1985',
 'Ley 29/1998',
 'Ley 4/2011',
 'Real Decreto 105/2008',
 'Real Decreto 922/2020',
 'Real Decreto 952/1997',
 'Real Decreto Legislativo 1/2001',
 'Ley 3/1995',
 'Resolución de 06 de febrero de 2021',
 'Ley 4/2013',
 'Ley 31/1995',
 'Ley 2/2020',
 'Ley 15/2018',
 'Real Decreto-ley 15/2018',
 'Ley 19/2013',
 'Real Decreto 289/2003',
 'Ley 22/2011',
 'Resolución de 18 de enero de 2021',
 'Real Decreto 1890/2008',
 'Real Decreto 128/2018',
 'Real Decreto Legislativo 5/2015',
 'Ley 2/1998',
 'Ley 9/2017',
 'Ley 40/2015',
 'Real Decreto Legislativo 2/2

## Importa BBDD ASECORP

In [713]:
ASECORP_BBDD = pd.read_csv('./ASECORP/ExportNormes_20210126.csv', delimiter=';')

In [714]:
ASECORP_BBDD

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
0,RES SLT/2313/2020 CAT,2313,"Resolución SLT/2313/2020, de 24 de septiembre,...",CAT,Cataluña,NaN,Prevención,[2]
1,(zzz) RES 12/4/2007 AST,12/4/2007,"Resolución de 12 abril de 2007, de la Consejer...",AST,Asturias,NaN,M. Ambiente,[1]
2,A CORUÑA 10/7/2015,10/7/2015,Modificación de la Ordenanza de vertidos y ser...,GAL,Galicia,A Coruña,M. Ambiente,[1]
3,A CORUÑA 11/8/2014 (1),11/8/2014,Ordenanza municipal reguladora de los procedim...,GAL,Galicia,A Coruña,M. Ambiente,[1]
4,A CORUÑA 11/8/2014 (2),11/8/2014,Ordenanza de protección contra la contaminació...,GAL,Galicia,A Coruña,M. Ambiente,[1]
...,...,...,...,...,...,...,...,...
19152,ZUMAIA 1/3/2001,1/3/2001,Ordenanza reguladora de la gestión del ciclo i...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19153,ZUMAIA 12/11/1999,12/11/1999,Normativa complementaria de las Normas Subsidi...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19154,ZUMAIA 15/6/2012,15/6/2012,Modificación de la Ordenanza municipal de la g...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19155,ZUMAIA 6/2/2004,6/2/2004,Modificación de la Normativa complementaria de...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]


In [715]:
ASECORP_BBDD_BOE = ASECORP_BBDD.loc[ASECORP_BBDD['Ambito'] == 'España']

In [716]:
ASECORP_BBDD_BOE

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
23,ACU 1/2020 ESP,1,Acuerdo Multilateral RID 1/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
35,ACU 11/6/2007 ESP,11/6/2007,Entrada en vigor del Acuerdo entre el Reino de...,ESP,España,NaN,M. Ambiente,[1]
57,ACU 17/10/2006 ESP,17/10/2006,Aplicación provisional del Acuerdo entre el Re...,ESP,España,NaN,M. Ambiente,[1]
71,ACU 19/11/2018 ESP,19/11/2018,Acuerdo de la Junta de Gobierno de la Confeder...,ESP,España,NaN,M. Ambiente,[1]
82,ACU 2/2020 ESP,2,Acuerdo Multilateral RID 2/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
...,...,...,...,...,...,...,...,...
18496,STC 86/2019 ESP,86,"Sentencia 86/2019, de 20 de junio de 2019. Rec...",ESP,España,NaN,M. Ambiente,[1]
18497,STC 87/2019 ESP,87,"Sentencia 87/2019, de 20 de junio de 2019. Rec...",ESP,España,NaN,M. Ambiente,[1]
18498,STC 88/2018 ESP,88,"Sentencia 88/2018, de 19 de julio de 2018. Con...",ESP,España,NaN,M. Ambiente / Reg. Industrial,[1][3]
18499,STC 9/3/2010 ESP,9/3/2010,"Sentencia de 9 de marzo de 2010, de la Sala Te...",ESP,España,NaN,M. Ambiente,[1]


In [717]:
# Crea nueva columna vacía de tipo lista en ASECORP_BBDD
ASECORP_BBDD_BOE['Tags'] = [[] for i in range(len(ASECORP_BBDD_BOE))]

In [718]:
titulo = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    titulo.append(str(row['Titulo']) + str(re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)))
    ASECORP_BBDD_BOE['Tags'][i] = re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)

In [719]:
ASECORP_BBDD_BOE[1200:1250]

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma,Tags
11629,ORD PRE/2827/2009 ESP,2827,"Orden PRE/2827/2009, de 19 de octubre, por la ...",ESP,España,NaN,M. Ambiente,[1],"[Orden PRE/2827/2009, Real Decreto 1370/2006]"
11630,ORD PRE/2843/2009 ESP,2843,"Orden PRE/2843/2009, de 19 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2843/2009, Real Decreto 2163/1994]"
11631,ORD PRE/2851/2010 ESP,2851,"Orden PRE/2851/2010, de 4 de noviembre, por la...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2851/2010, Real Decreto 2163/1994]"
11632,ORD PRE/2871/2011 ESP,2871,"Orden PRE/2871/2011, de 25 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2871/2011, Real Decreto 2163/1994]"
11633,ORD PRE/2872/2011 ESP,2872,"Orden PRE/2872/2011, de 25 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2872/2011, Real Decreto 2163/1994]"
11634,ORD PRE/29/2004 ESP,29,"Orden PRE/29/2004, de 15 de enero, por la que ...",ESP,España,NaN,M. Ambiente,[1],"[Orden PRE/29/2004, Real Decreto 837/2002]"
11635,ORD PRE/2922/2005 ESP,2922,"Orden PRE/2922/2005, de 19 de septiembre, por ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2922/2005, Orden de 8 de marzo de 1..."
11636,ORD PRE/2957/2008 ESP,2957,"Orden PRE/2957/2008, de 10 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2957/2008, Real Decreto 2163/1994]"
11637,ORD PRE/3/2006 ESP,3,"Orden PRE/3/2006, de 12 de enero, por la que s...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/3/2006, Real Decreto 255/2003]"
11638,ORD PRE/3159/2004 ESP,3159,"Orden PRE/3159/2004, de 28 de septiembre, por ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/3159/2004, Real Decreto 1406/1989]"


In [720]:
# titulo

In [721]:
len(titulo)

2959

In [722]:
## Busca patrones regex definidos en columna títulos del DF 
boletin = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    boletin.append(str(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)))

In [723]:
boletin[0:25]

['[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "['Acuerdo Multilateral M-150']",
 "['Acuerdo Multilateral M-159']",
 "['Acuerdo multilateral M-168']",
 "['Acuerdo Multilateral M-170']",
 "['Acuerdo Multilateral M-171']",
 "['Acuerdo Multilateral M-173']",
 "['Acuerdo Multilateral M-175']",
 '[]',
 "['Acuerdo Multilateral M-178']",
 "['Acuerdo Multilateral M-198']"]

In [724]:
## Para cada fila de la BBDD recoge la expresión REGEX encontrada y si no existe no la incluye 
## en la lista resultante llamada 'boletin'. Además si no detecta expresión la cuenta como vacía
## en variable 'n_vacios', y la añade a la lista 'vacios' para inspeccionar posteriormente 
boletin = []
vacios = []
n_vacios = 0
for i, row in ASECORP_BBDD_BOE.iterrows():
    regex_result = re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)
    if len(regex_result) != 0:
        boletin.append(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE))
    else:
        n_vacios += 1
        vacios.append(row['Titulo'])

In [725]:
n_vacios

276

In [726]:
vacios[0:25]

['Acuerdo Multilateral RID 1/2020 en virtud de la Sección 1.5.1 del Reglamento del Transporte Internacional de Mercancías Peligrosas por Ferrocarril (RID), relativo a los certificados de consejeros de seguridad de conformidad con el punto 1.8.3.7 del RID, hecho en Madrid el 25 de marzo de 2020',
 'Entrada en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Aplicación provisional del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Acuerdo de la Junta de Gobierno de la Confederación Hidrográfica del Segura celebrada el día 30 de octubre de 2018 sobre aplicación de indisponibilidad o reducción de dotaciones de acuerdo al Plan Especial de Sequía y al Real Decreto que la regula',
 'Acuerdo Multilateral RID 2/2020 en virtud de la Sección 1.5.1 del Reglamento del Transporte Internacional de Merc

In [727]:
boletin[0:25]

[['Acuerdo Multilateral M-150'],
 ['Acuerdo Multilateral M-159'],
 ['Acuerdo multilateral M-168'],
 ['Acuerdo Multilateral M-170'],
 ['Acuerdo Multilateral M-171'],
 ['Acuerdo Multilateral M-173'],
 ['Acuerdo Multilateral M-175'],
 ['Acuerdo Multilateral M-178'],
 ['Acuerdo Multilateral M-198'],
 ['Acuerdo Multilateral M-214'],
 ['Acuerdo Multilateral M-218'],
 ['Acuerdo Multilateral M-226'],
 ['Acuerdo Multilateral M-228'],
 ['Acuerdo Multilateral M-231'],
 ['Acuerdo Multilateral M-236'],
 ['Acuerdo Multilateral M-245'],
 ['Acuerdo Multilateral M-254'],
 ['Acuerdo Multilateral M-259'],
 ['Acuerdo Multilateral M-265'],
 ['Acuerdo Multilateral M-268'],
 ['Acuerdo Multilateral M-271'],
 ['Acuerdo Multilateral M-273'],
 ['Acuerdo Multilateral M-276'],
 ['Acuerdo Multilateral M-291'],
 ['Acuerdo Multilateral M-292']]

In [728]:
# Flatten list of lists
boletin_ASECORP_flat_list = [item for sublist in boletin for item in sublist]

In [729]:
boletin_ASECORP_flat_list[0:25]

['Acuerdo Multilateral M-150',
 'Acuerdo Multilateral M-159',
 'Acuerdo multilateral M-168',
 'Acuerdo Multilateral M-170',
 'Acuerdo Multilateral M-171',
 'Acuerdo Multilateral M-173',
 'Acuerdo Multilateral M-175',
 'Acuerdo Multilateral M-178',
 'Acuerdo Multilateral M-198',
 'Acuerdo Multilateral M-214',
 'Acuerdo Multilateral M-218',
 'Acuerdo Multilateral M-226',
 'Acuerdo Multilateral M-228',
 'Acuerdo Multilateral M-231',
 'Acuerdo Multilateral M-236',
 'Acuerdo Multilateral M-245',
 'Acuerdo Multilateral M-254',
 'Acuerdo Multilateral M-259',
 'Acuerdo Multilateral M-265',
 'Acuerdo Multilateral M-268',
 'Acuerdo Multilateral M-271',
 'Acuerdo Multilateral M-273',
 'Acuerdo Multilateral M-276',
 'Acuerdo Multilateral M-291',
 'Acuerdo Multilateral M-292']

In [730]:
## Elimina duplicados
boletin_ASECORP_flat_list = list(set(boletin_ASECORP_flat_list))

In [731]:
boletin_ASECORP_flat_list[0:25]

['Orden CTE/2723/2002',
 'Orden de 11 de Octubre de 1988',
 'Real Decreto 244/2016',
 'Resolución de 17 de marzo de 2011',
 'Circular 3/2018',
 'Orden TEC/1171/2018',
 'Ley 9/2003',
 'Real Decreto 335/2016',
 'Real Decreto 128/2013',
 'Real Decreto-ley 6/2009',
 'Resolución de 9 de octubre de 2009',
 'Orden PRE/252/2006',
 'Real Decreto 1311/2012',
 'Real Decreto 178/2004',
 'Real Decreto 102/2011',
 'Real Decreto 777/2006',
 'Orden TEC/427/2019',
 'Orden de 7 de marzo de 1981',
 'Real Decreto 451/2020',
 'ley 34/1992',
 'Resolución de 26 de marzo de 2020',
 'Ley 6/2001',
 'Real Decreto 237/2000',
 'Ley 43/2003',
 'Real Decreto 201/2012']

In [732]:
## Busca coincidencias entre lista boletines BOEs explorados y lista boletines de BBDD ASECORP
set(boletin_flat_list) & set(boletin_ASECORP_flat_list)

{'Ley 21/1992',
 'Ley 21/2013',
 'Ley 22/2011',
 'Ley 24/2013',
 'Ley 30/1992',
 'Ley 31/1995',
 'Ley 34/2007',
 'Ley 37/2003',
 'Ley 9/2003',
 'Ley 9/2017',
 'Real Decreto 105/2008',
 'Real Decreto 110/2015',
 'Real Decreto 1890/2008',
 'Real Decreto 212/2002',
 'Real Decreto 524/2006',
 'Real Decreto 952/1997',
 'Real Decreto Legislativo 1/2001',
 'Real Decreto Legislativo 2/2015',
 'Real Decreto-ley 15/2018'}

In [733]:
#DOCM_sumarios['Tags'].isin(ASECORP_BBDD_BOE['Tags'])
for row_to_compare in DOCM_sumarios['Tags']:
    for row_comparing in ASECORP_BBDD_BOE['Tags']:
        if set(row_comparing) & set(row_to_compare):
            print(set(row_comparing) & set(row_to_compare))

{'Ley 31/1995'}
{'Ley 31/1995'}
{'Real Decreto Legislativo 2/2015'}
{'Ley 31/1995'}
{'Ley 31/1995'}
{'Ley 9/2017'}
{'Ley 30/1992'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 9/2003'}
{'Ley 9/2003'}
{'Ley 9/2003'}
{'Ley 9/2003'}
{'Ley 9/2003'}
{'Ley 9/2003'}
{'Ley 9/2003'}
{'Ley 9/2003'}
{'Ley 9/2003'}
{'Ley 9/2003'}
{'Real Decreto Legislativo 1/2001'}
{'Real Decreto Legislativo 1/2001'}
{'Real Decreto Legislativo 1/2001'}
{'Real Decreto Legislativo 1/2001'}
{'Real Decreto Legislativo 1/2001'}
{'Real Decreto Legislativo 1/2001'}
{'Real Decreto Legislativo 1/2001'}
{'Ley 24/2013'}
{'Ley 24/2013'}
{'Real Decreto-ley 15/2018'}
{'Real Decreto-ley 15/2018'}
{'Real Decreto Legislativo 1/2001'}
{'Ley 21/2013'}
{'Ley 22/2011'}
{'Ley 31/1995'}
{'Ley 34/2007'}
{'Ley 37/2003'}
{'Ley 22/2011'}
{'Ley 21/2013'}
{'Ley 22/2011'}
{'Real Decreto Legislativo 1/2001'}
{'Ley 22/2011'}
{

In [734]:
for i, row_to_compare in DOCM_sumarios.iterrows():
    for j, row_comparing in ASECORP_BBDD_BOE.iterrows():
        if set(row_to_compare['Tags']) & set(row_comparing['Tags']):
            DOCM_sumarios['Match_ASECORP_BBDD'][i].append (ASECORP_BBDD_BOE['Codigo'][j])
            print(str(set(row_to_compare['Tags']) & set(row_comparing['Tags'])) + ' ' + str(row_comparing['Codigo']))

{'Ley 31/1995'} LEY 31/1995 ESP
{'Ley 31/1995'} RD 171/2004 ESP
{'Real Decreto Legislativo 2/2015'} RDG 2/2015 ESP
{'Ley 31/1995'} RES 29/5/2017 ESP
{'Ley 31/1995'} RES 5/11/2010 ESP
{'Ley 9/2017'} LEY 9/2017 ESP
{'Ley 30/1992'} RD 1771/1994 ESP
{'Ley 22/2011'} LEY 22/2011 ESP
{'Ley 22/2011'} LEY 5/2013 ESP
{'Ley 22/2011'} ORD AAA/699/2016 ESP
{'Ley 22/2011'} ORD APM/189/2018 ESP
{'Ley 22/2011'} ORD APM/205/2018 ESP
{'Ley 22/2011'} ORD APM/206/2018 ESP
{'Ley 22/2011'} ORD APM/397/2018 ESP
{'Ley 22/2011'} ORD TEC/852/2019 ESP
{'Ley 22/2011'} ORD TED/363/2020 ESP
{'Ley 22/2011'} ORD TED/426/2020 ESP
{'Ley 9/2003'} LEY 9/2003 ESP
{'Ley 9/2003'} RD 178/2004 ESP
{'Ley 9/2003'} RD 191/2013 ESP
{'Ley 9/2003'} RD 364/2017 ESP
{'Ley 9/2003'} RD 452/2019 ESP
{'Ley 9/2003'} LEY 9/2003 ESP
{'Ley 9/2003'} RD 178/2004 ESP
{'Ley 9/2003'} RD 191/2013 ESP
{'Ley 9/2003'} RD 364/2017 ESP
{'Ley 9/2003'} RD 452/2019 ESP
{'Real Decreto Legislativo 1/2001'} LEY 1/2018 ESP
{'Real Decreto Legislativo 1/2001'} 

In [744]:
DOCM_sumarios

,item_Title,item_urlHTML,item_urlPDF,NID,numero_diario,numero_pagina,rango,organo_emisor,Tags,Match_ASECORP_BBDD,Fecha_publicacion
0,Sanidad. Procedimiento Ordinario 119/2021. [NI...,https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1372,29/2021,5362,Edicto,Tribunal Superior de Justicia de Castilla-La M...,"[Resolución de 28 de enero de 2021, Resolución...",[],2021-02-12
1,"Régimen Local. Resolución de 05/02/2021, de la...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1231,29/2021,5364,Resolución,Viceconsejería de Administración Local y Coord...,"[Ley 7/1985, Real Decreto 128/2018, Real Decre...",[],2021-02-12
2,"Función Pública. Resolución de 02/02/2021, de ...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1109,29/2021,5365,Resolución,Dirección General de la Función Pública,"[Ley 40/2015, Ley 31/1995, Ley 12/2010, Real D...","[LEY 31/1995 ESP, RD 171/2004 ESP, RDG 2/2015 ...",2021-02-12
3,"Formación. Resolución de 05/02/2021, de la Dir...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1248,29/2021,5380,Resolución,Dirección General de la Función Pública,"[Ley 4/2011, Ley 40/2015, Ley 39/2015, Ley 40/...","[LEY 9/2017 ESP, RD 1771/1994 ESP]",2021-02-12
4,"Convenios. Resolución de 05/02/2021, de la Sec...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1250,29/2021,5504,Resolución,Secretaría General de la Presidencia,"[Ley 19/2013, Ley 19/2013, Ley 40/2015, Ley 40...",[],2021-02-12
...,...,...,...,...,...,...,...,...,...,...,...
63,"Anuncio de 01/02/2021, del Ayuntamiento de Poz...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1122,29/2021,5595,Anuncio,Ayuntamiento de Pozohondo (Albacete),[],[],2021-02-12
64,"Anuncio de 29/01/2021, del Ayuntamiento de La ...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1154,29/2021,5596,Anuncio,Ayuntamiento de La Villa de Don Fadrique (Toledo),[],[],2021-02-12
65,"Anuncio de 05/02/2021, de la Empresa Pública G...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1202,29/2021,5597,Anuncio,Empresa Pública Gestión Ambiental de Castilla-...,[],[],2021-02-12
66,"Anuncio de 05/02/2021, de la Empresa Pública G...",https://docm.jccm.es/portaldocm/detalleDocumen...,https://docm.jccm.es/portaldocm/descargarArchi...,2021/1203,29/2021,5598,Anuncio,Empresa Pública Gestión Ambiental de Castilla-...,[],[],2021-02-12


In [736]:
DOCM_sumarios['Match_ASECORP_BBDD'][2]

['LEY 31/1995 ESP',
 'RD 171/2004 ESP',
 'RDG 2/2015 ESP',
 'RES 29/5/2017 ESP',
 'RES 5/11/2010 ESP']

In [737]:
DOCM_sumarios['Tags'][2]

['Ley 40/2015',
 'Ley 31/1995',
 'Ley 12/2010',
 'Real Decreto Legislativo 5/2015',
 'Ley 4/2011',
 'Real Decreto Legislativo 2/2015',
 'Ley 4/2011',
 'Ley 12/2010',
 'Ley 12/2010',
 'Ley 12/2010',
 'Ley 31/1995',
 'Ley 12/2010',
 'Ley 31/1995',
 'Ley 40/2015',
 'Ley 40/2015']

# Genera Fichero EXCEL de resultados

In [837]:
DOCM_sumarios_final = DOCM_sumarios

In [838]:
## Cambia orden de columnas y elimina las no necesarias  
DOCM_sumarios_final.rename(columns={'NID': 'Item_id', 'item_Title': 'Item_Title', 'item_urlPDF': 'PDF_Link'}, inplace=True)
DOCM_sumarios_final = DOCM_sumarios[['Item_id','Item_Title','PDF_Link','Fecha_publicacion','Tags','Match_ASECORP_BBDD']]

In [839]:
## Crea función que convierte lista a string en todas las columnas de DOCM_sumarios
## para evitar en presentación final los caracteres [' '] propios de las listas
def list2Str(lst):
    if type(lst) is list: # apply conversion to list columns
        return", ".join(lst)
    else:
        return lst

In [840]:
## Aplica función de conversión de listas a strings
DOCM_sumarios_final = DOCM_sumarios_final.apply(lambda x: [list2Str(i) for i in x])

In [841]:
  DOCM_sumarios_final[0:10]

,Item_id,Item_Title,PDF_Link,Fecha_publicacion,Tags,Match_ASECORP_BBDD
0,2021/1372,Sanidad. Procedimiento Ordinario 119/2021. [NI...,https://docm.jccm.es/portaldocm/descargarArchi...,2021-02-12,"Resolución de 28 de enero de 2021, Resolución ...",
1,2021/1231,"Régimen Local. Resolución de 05/02/2021, de la...",https://docm.jccm.es/portaldocm/descargarArchi...,2021-02-12,"Ley 7/1985, Real Decreto 128/2018, Real Decret...",
2,2021/1109,"Función Pública. Resolución de 02/02/2021, de ...",https://docm.jccm.es/portaldocm/descargarArchi...,2021-02-12,"Ley 40/2015, Ley 31/1995, Ley 12/2010, Real De...","LEY 31/1995 ESP, RD 171/2004 ESP, RDG 2/2015 E..."
3,2021/1248,"Formación. Resolución de 05/02/2021, de la Dir...",https://docm.jccm.es/portaldocm/descargarArchi...,2021-02-12,"Ley 4/2011, Ley 40/2015, Ley 39/2015, Ley 40/2...","LEY 9/2017 ESP, RD 1771/1994 ESP"
4,2021/1250,"Convenios. Resolución de 05/02/2021, de la Sec...",https://docm.jccm.es/portaldocm/descargarArchi...,2021-02-12,"Ley 19/2013, Ley 19/2013, Ley 40/2015, Ley 40/...",
5,2021/1194,"Notificaciones. Notificación de 05/02/2021, de...",https://docm.jccm.es/portaldocm/descargarArchi...,2021-02-12,"Ley 39/2015, Ley 39/2015, Ley 39/2015, Ley 29/...",
6,2021/1236,"Sanciones. Notificación de 14/01/2021, de la D...",https://docm.jccm.es/portaldocm/descargarArchi...,2021-02-12,"Ley 39/2015, Ley 39/2015",
7,2021/1237,"Sanciones. Notificación de 14/01/2021, de la D...",https://docm.jccm.es/portaldocm/descargarArchi...,2021-02-12,"Ley 39/2015, Ley 39/2015",
8,2021/1238,"Sanciones. Notificación de 14/01/2021, de la D...",https://docm.jccm.es/portaldocm/descargarArchi...,2021-02-12,"Ley 39/2015, Ley 39/2015",
9,2021/1239,"Sanciones. Notificación de 14/01/2021, de la D...",https://docm.jccm.es/portaldocm/descargarArchi...,2021-02-12,"Ley 39/2015, Ley 39/2015",


In [842]:
## Generar hyperlink a artículo BOE en CSV "=HYPERLINK("https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf";"BOE-A-2021-1474")"
## https://www.boe.es/diario_boe/xml.php?id=## https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf

DOCM_sumarios_final_CSV = DOCM_sumarios_final

In [843]:
DOCM_sumarios_final_CSV['Item_id'] = '=HIPERVINCULO(' + '"' + DOCM_sumarios_final_CSV['PDF_Link'] + '";' \
                                                        + '"' + DOCM_sumarios_final_CSV['Item_id'] + '")'                                                        

In [844]:
print(DOCM_sumarios_final_CSV['Item_id'][0])

=HIPERVINCULO("https://docm.jccm.es/portaldocm/descargarArchivo.do?ruta=2021/02/12/pdf/2021_1372.pdf&tipo=rutaDocm";"2021/1372")


In [845]:
# Elimina columna PDF_Link
DOCM_sumarios_final_CSV = DOCM_sumarios_final_CSV[['Item_id','Item_Title','Fecha_publicacion','Tags','Match_ASECORP_BBDD']]

In [846]:
DOCM_sumarios_final_CSV

,Item_id,Item_Title,Fecha_publicacion,Tags,Match_ASECORP_BBDD
0,"=HIPERVINCULO(""https://docm.jccm.es/portaldocm...",Sanidad. Procedimiento Ordinario 119/2021. [NI...,2021-02-12,"Resolución de 28 de enero de 2021, Resolución ...",
1,"=HIPERVINCULO(""https://docm.jccm.es/portaldocm...","Régimen Local. Resolución de 05/02/2021, de la...",2021-02-12,"Ley 7/1985, Real Decreto 128/2018, Real Decret...",
2,"=HIPERVINCULO(""https://docm.jccm.es/portaldocm...","Función Pública. Resolución de 02/02/2021, de ...",2021-02-12,"Ley 40/2015, Ley 31/1995, Ley 12/2010, Real De...","LEY 31/1995 ESP, RD 171/2004 ESP, RDG 2/2015 E..."
3,"=HIPERVINCULO(""https://docm.jccm.es/portaldocm...","Formación. Resolución de 05/02/2021, de la Dir...",2021-02-12,"Ley 4/2011, Ley 40/2015, Ley 39/2015, Ley 40/2...","LEY 9/2017 ESP, RD 1771/1994 ESP"
4,"=HIPERVINCULO(""https://docm.jccm.es/portaldocm...","Convenios. Resolución de 05/02/2021, de la Sec...",2021-02-12,"Ley 19/2013, Ley 19/2013, Ley 40/2015, Ley 40/...",
...,...,...,...,...,...
63,"=HIPERVINCULO(""https://docm.jccm.es/portaldocm...","Anuncio de 01/02/2021, del Ayuntamiento de Poz...",2021-02-12,,
64,"=HIPERVINCULO(""https://docm.jccm.es/portaldocm...","Anuncio de 29/01/2021, del Ayuntamiento de La ...",2021-02-12,,
65,"=HIPERVINCULO(""https://docm.jccm.es/portaldocm...","Anuncio de 05/02/2021, de la Empresa Pública G...",2021-02-12,,
66,"=HIPERVINCULO(""https://docm.jccm.es/portaldocm...","Anuncio de 05/02/2021, de la Empresa Pública G...",2021-02-12,,


In [847]:
DOCM_sumarios_final_CSV.to_csv("./ASECORP/Resultados_Matching_DOCM_" + today.strftime("%Y%m%d") + ".csv" ,index=False) 